# Organise model and observational data 
- I am using the conda environment version 22.04 http://climate-cms.wikis.unsw.edu.au/Conda#22.04



### In this notebook: 
- Pull out required model fields (bx400 = control, cg207 = 'ice' experiment) and organise to match observations. 
- Pull out already processed observation data and organise:  
    - MODIS data has been downloaded from: https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MCD06COSP_D3_MODIS/ 
    - and then processed using this code  https://github.com/sfiddes/code_for_publications_2022/blob/main/ACCESS_cloud_radiation_eval/pre-processing.ipynb 
    - which was generated for Fiddes et al. 2022 (ACPD https://doi.org/10.5194/acp-2022-259)

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import xarray as xr

# ACCESS-AM2 Model 

In [2]:
jobname = 'bx400' # or cg207

In [3]:
fname = '/g/data/jk72/slf563/ACCESS/output/{}/daily/{}a.pdc2*'.format(jobname,jobname)
data = xr.open_mfdataset(fname,parallel=True)
#LWP_A = xr.DataArray((data['field2466']/data['field2330'])*1000,name='LWP') # convert from kg/m2 to g/m2
#IWP_A = xr.DataArray((data['field2467']/data['field2330'])*1000,name='IWP') # convert from kg/m2 to g/m2
CFL_A = xr.DataArray(data['field2452']/data['field2330'],name='CFL') 
CFI_A = xr.DataArray(data['field2453']/data['field2330'],name='CFI')
TauL_A = xr.DataArray(data['field2458']/data['field2452'],name='TauL')
TauI_A = xr.DataArray(data['field2459']/data['field2453'],name='TauI')
CTP_A = xr.DataArray((data['field2465']/data['field2451'])/100,name='CTP') # convert to hPa

In [4]:
fname = '/g/data/jk72/slf563/ACCESS/output/{}/daily/{}a.pd.glob20*'.format(jobname,jobname)
data_met = xr.open_mfdataset(fname,parallel=True)
CRESWT_A = xr.DataArray(data_met.rsutcs-data_met.rsut,name='SWCREtoa')
SWT_A = xr.DataArray(data_met.rsut,name='SWtoa')
# note I have used the raw model LWP/IWP not the COSP versions due to unrealistically large biases. See Fiddes et al. (2022) for details. 
LWP_A = xr.DataArray(data_met.lwp,name='LWP')*1000 # convert from kg/m2 to g/m2
IWP_A = xr.DataArray(data_met.clivi,name='IWP')*1000 # convert from kg/m2 to g/m2

In [5]:
# check preip rates to see if cloud life time may have changed?
prl = xr.DataArray(data_met.prrn,name='rain')*60*60*24 # kg m-2 s-1 to mm/day
prs = xr.DataArray(data_met.prsn,name='snow')*60*60*24 # kg m-2 s-1 to mm/day
tmp = xr.merge([prl,prs])
tmp.to_netcdf('/g/data/jk72/slf563/ACCESS/ML_data/precip_vars_{}_2015-2019.nc'.format(jobname))

In [6]:
mod = xr.merge([LWP_A,IWP_A,CFL_A,CFI_A,TauL_A,TauI_A,CTP_A,CRESWT_A,SWT_A])
mod.to_netcdf('/g/data/jk72/slf563/ACCESS/ML_data/COSP_vars_{}_2015-2019.nc'.format(jobname))
data.close()
data_met.close()

In [7]:
# obs 

In [8]:
fdir = '/g/data/p66/slf563/OBS/MCD06COSP_D3_MODIS/MCD06COSP_D3_MODIS.201*cloud.nc'
MODIS = xr.open_mfdataset(fdir)

In [9]:
MODIS = MODIS[['CTP','LWP','IWP','CFI','CFL','TauL','TauI']]
MODIS['LWP'] = MODIS['LWP']*MODIS['CFL'] # Multiply by the cloud fraction to match the access data
MODIS['IWP'] = MODIS['IWP']*MODIS['CFI'] # Multiply by the cloud fraction to match the access data

In [10]:
fdir = '/g/data/jk72/slf563/OBS/CERES_SYN1D/CERES_SYN1deg-Day_Terra-Aqua-MODIS_Ed4.1_Subset_*.nc'
CERES = xr.open_mfdataset(fdir,parallel=True)
CERES = CERES.sel(time=slice('20150101','20191231'))
CERES = CERES.interp(lat=mod.lat,lon=mod.lon)

CRESWT_M = xr.DataArray(CERES.toa_sw_clr_daily - CERES.toa_sw_all_daily,name='SWCREtoa')
SWT_M = xr.DataArray(CERES.toa_sw_all_daily,name='SWtoa')

In [11]:
obs = xr.merge([MODIS,CRESWT_M,SWT_M,])
obs = obs.transpose('time','lat','lon')
obs.to_netcdf('/g/data/jk72/slf563/ACCESS/ML_data/COSP_vars_MODIS_2015-2019.nc')